## 1. 구글 드라이브에서 마운틴하기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


## 2. 파일 경로 지정하기

진짜 음성 데이터가 담긴 폴더: /content/drive/MyDrive/Colab Notebooks/wav2vec classification/real

가짜 음성 데이터가 담긴 폴더: /content/drive/MyDrive/Colab Notebooks/wav2vec classification/fake

본인의 구글 드라이브에 저장되어 있는 폴더로 수정해야 함

## 3. 데이터셋 미리 생성

In [ ]:
# preprocess_dataset.py
!pip install fsspec==2024.10.0
!pip install transformers datasets
from datasets import Dataset
import torchaudio
from transformers import Wav2Vec2Processor
import os
from sklearn.model_selection import train_test_split
from datasets import load_dataset
import torch

# 경로 설정 (각자 구글 드라이브에 저장되어 있는 폴더로 수정해야 함)
real_path = "/content/drive/MyDrive/Colab Notebooks/wav2vec classification/real"
fake_path = "/content/drive/MyDrive/Colab Notebooks/wav2vec classification/fake"

# 데이터 로딩 함수
def load_data(folder_path, label, max_files=None):
    files = []
    labels = []
    for i, file in enumerate(os.listdir(folder_path)):
        if file.endswith('.wav'):
            files.append(os.path.join(folder_path, file))
            labels.append(label)
        if max_files is not None and i+1 >= max_files:
            break
    return files, labels

# 필요한 경우 max_files 조정
half = 25000

real_files, real_labels = load_data(real_path, 1, max_files=half)
fake_files, fake_labels = load_data(fake_path, 0, max_files=half)

# 모든 데이터를 사용하고 싶으면 위의 두 줄 코드를 아래 두 줄 코드로 변경해야 함
# real_files, real_labels = load_data(real_path, 1)
# fake_files, fake_labels = load_data(fake_path, 0)

all_files = real_files + fake_files
all_labels = real_labels + fake_label

train_files, test_files, train_labels, test_labels = train_test_split(all_files, all_labels, test_size=0.2, random_state=42)

model_name = "facebook/wav2vec2-base"
processor = Wav2Vec2Processor.from_pretrained(model_name)

# 전처리 함수
def preprocess(file_path):
    speech, sample_rate = torchaudio.load(file_path)
    # 이미 16kHz로 맞춰져 있다고 가정
    inputs = processor(speech.squeeze().numpy(), sampling_rate=16000, return_tensors="pt", padding=True)
    return {
        'input_values': inputs['input_values'][0],  # (batch 차원 제거)
    }

# train 데이터셋
train_data = {'file_path': train_files, 'label': train_labels}
train_dataset = Dataset.from_dict(train_data)
train_dataset = train_dataset.map(lambda example: {
    **preprocess(example['file_path']),
    'label': example['label']
})

# test 데이터셋
test_data = {'file_path': test_files, 'label': test_labels}
test_dataset = Dataset.from_dict(test_data)
test_dataset = test_dataset.map(lambda example: {
    **preprocess(example['file_path']),
    'label': example['label']
})

# 전처리 완료된 dataset 저장
# 원하시는 경로로 변경 가능
train_dataset.save_to_disk("/content/drive/MyDrive/Colab Notebooks/wav2vec classification/train_dataset")
test_dataset.save_to_disk("/content/drive/MyDrive/Colab Notebooks/wav2vec classification/test_dataset")
